# Iris 데이터 딥러닝 실험


### import, iris.csv 데이터 처리

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

df = pd.read_csv("data/iris.csv", encoding="cp949")
df = df.drop(columns=["Id"])
df = df.rename(columns={"SepalLengthCm":"sepal_length",
                        "SepalWidthCm":"sepal_width",
                        "PetalLengthCm":"petal_length",
                        "PetalWidthCm":"petal_width",
                        "Species":"class"})



### One-Hot Encoding, 학습/검증 데이터 나누기

In [9]:
df = pd.get_dummies(df, columns=["class"], dtype="int")

x = df.iloc[:, :4]
y = df.iloc[:, 4:]

print(x.head())
print(y.head())

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, stratify=y) # random_state=123

print(x_train.head())
print(y_train.head())


print(y_train.value_counts())
print(y_test.value_counts())

   sepal_length  sepal_width  petal_length  petal_width
0           5.1          3.5           1.4          0.2
1           4.9          3.0           1.4          0.2
2           4.7          3.2           1.3          0.2
3           4.6          3.1           1.5          0.2
4           5.0          3.6           1.4          0.2
   class_Iris-setosa  class_Iris-versicolor  class_Iris-virginica
0                  1                      0                     0
1                  1                      0                     0
2                  1                      0                     0
3                  1                      0                     0
4                  1                      0                     0
     sepal_length  sepal_width  petal_length  petal_width
38            4.4          3.0           1.3          0.2
92            5.8          2.6           4.0          1.2
2             4.7          3.2           1.3          0.2
9             4.9          3.1      

### Model(1) 설정

In [10]:
inputs = keras.Input(shape=(4,))
x = layers.Dense(1, activation="linear")(inputs)

outputs = layers.Dense(3, activation="softmax")(x)
model1 = keras.Model(inputs=inputs, outputs=outputs)

model1.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 4)]               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 5         
                                                                 
 dense_3 (Dense)             (None, 3)                 6         
                                                                 
Total params: 11
Trainable params: 11
Non-trainable params: 0
_________________________________________________________________


#### ModelCheckpoint callback function

In [11]:
checkpoint = keras.callbacks.ModelCheckpoint(filepath="model/model1.hdf5", monitor="val_accuracy", save_best_only=True, verbose=1)

#### 모델 컴파일, 실행

In [12]:
model1.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
history1 = model1.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=200, batch_size=5, callbacks=[checkpoint])

Epoch 1/200


 1/18 [>.............................] - ETA: 16s - loss: 2.0484 - accuracy: 0.2000
Epoch 1: val_accuracy improved from -inf to 0.33333, saving model to model\model1.hdf5
18/18 [==============================] - 1s 23ms/step - loss: 2.4076 - accuracy: 0.3333 - val_loss: 2.3482 - val_accuracy: 0.3333
Epoch 2/200
 1/18 [>.............................] - ETA: 0s - loss: 2.0037 - accuracy: 0.0000e+00
Epoch 2: val_accuracy did not improve from 0.33333
18/18 [==============================] - 0s 7ms/step - loss: 2.2367 - accuracy: 0.3333 - val_loss: 2.1841 - val_accuracy: 0.3333
Epoch 3/200
 1/18 [>.............................] - ETA: 0s - loss: 1.7615 - accuracy: 0.6000
Epoch 3: val_accuracy did not improve from 0.33333
18/18 [==============================] - 0s 7ms/step - loss: 2.0807 - accuracy: 0.3333 - val_loss: 2.0325 - val_accuracy: 0.3333
Epoch 4/200
 1/18 [>.............................] - ETA: 0s - loss: 2.1408 - accuracy: 0.4000
Epoch 4: val_accuracy did not improve from 0.33333

#### Model(1) 정확도

In [13]:
print(min(history1.history["loss"]))
print(max(history1.history["accuracy"]))
print(min(history1.history["val_loss"]))
print(max(history1.history["val_accuracy"]))

0.2508279085159302
0.9666666388511658
0.23696652054786682
0.9666666388511658


### 저장된 best model Load

In [14]:
model1_best = keras.models.load_model("model/model1.hdf5")
model1_best.evaluate(x_test, y_test)

2/2 [==============================] - 0s 6ms/step - loss: 0.2495 - accuracy: 0.9667


[0.24949152767658234, 0.9666666388511658]

---

### Model(2) 설정

In [15]:
inputs = keras.Input(shape=(4,))
x = layers.Dense(12, activation ="relu")(inputs)
x = layers.Dense(8, activation ="relu")(x)

outputs = layers.Dense(3, activation="softmax")(x)
model2 = keras.Model(inputs = inputs, outputs = outputs)

model2.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 4)]               0         
                                                                 
 dense_4 (Dense)             (None, 12)                60        
                                                                 
 dense_5 (Dense)             (None, 8)                 104       
                                                                 
 dense_6 (Dense)             (None, 3)                 27        
                                                                 
Total params: 191
Trainable params: 191
Non-trainable params: 0
_________________________________________________________________


#### Model Checkpoint Callback function

In [16]:
checkpoint = keras.callbacks.ModelCheckpoint(filepath="model/model2.hdf5", monitor="val_accuracy", save_best_only=True, verbose=1)

#### 모델 컴파일, 실행

In [17]:
model2.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
history2 = model2.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=200, batch_size=5, callbacks=[checkpoint])

Epoch 1/200
 1/18 [>.............................] - ETA: 18s - loss: 1.4401 - accuracy: 0.2000
Epoch 1: val_accuracy improved from -inf to 0.36667, saving model to model\model2.hdf5
18/18 [==============================] - 1s 19ms/step - loss: 1.2924 - accuracy: 0.3333 - val_loss: 1.2501 - val_accuracy: 0.3667
Epoch 2/200
 1/18 [>.............................] - ETA: 0s - loss: 1.2833 - accuracy: 0.2000
Epoch 2: val_accuracy improved from 0.36667 to 0.43333, saving model to model\model2.hdf5
18/18 [==============================] - 0s 7ms/step - loss: 1.2340 - accuracy: 0.3889 - val_loss: 1.1984 - val_accuracy: 0.4333
Epoch 3/200
 1/18 [>.............................] - ETA: 0s - loss: 1.2109 - accuracy: 0.4000
Epoch 3: val_accuracy improved from 0.43333 to 0.66667, saving model to model\model2.hdf5
18/18 [==============================] - 0s 8ms/step - loss: 1.1845 - accuracy: 0.5556 - val_loss: 1.1419 - val_accuracy: 0.6667
Epoch 4/200
 1/18 [>.............................] - ETA: 0

#### Model(2) 정확도

In [18]:
print(min(history2.history["loss"]))
print(max(history2.history["accuracy"]))
print(min(history2.history["val_loss"]))
print(max(history2.history["val_accuracy"]))

0.08576779067516327
0.9666666388511658
0.045866090804338455
1.0


#### 저장된 best model Load

In [19]:
model2_best = keras.models.load_model("model/model2.hdf5")
model2_best.evaluate(x_test, y_test)

2/2 [==============================] - 0s 5ms/step - loss: 0.2498 - accuracy: 1.0000


[0.24976147711277008, 1.0]

---

### 최종 정확도 비교

In [20]:
print(history1.history["val_accuracy"][-1])
print(history2.history["val_accuracy"][-1])

0.949999988079071
1.0


### best model 정확도 (최대 정확도) 비교

In [21]:
print(max(history1.history["val_accuracy"]))
print(max(history2.history["val_accuracy"]))

0.9666666388511658
1.0
